In [1]:
%matplotlib inline

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [3]:
# Tensorflow Limit GPU usage
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

assert train_images.shape == (50000, 32, 32, 3)
assert test_images.shape == (10000, 32, 32, 3)
assert train_labels.shape == (50000, 1)
assert test_labels.shape == (10000, 1)

In [5]:
# Dimension constants
im_ht = 32
im_wt = 32

kr_ht = 4
kr_wt = 4

no_ch = 3
no_im = 10
no_kr = 5

y_ht = im_ht - kr_ht + 1
y_wt = im_wt - kr_wt + 1

no_of_patches = y_ht*y_wt
patch_len = kr_ht*kr_wt*no_ch

In [6]:
# Numpy Random Number Generator
rng = np.random.default_rng(20220525)

In [7]:
# [filter_height, filter_width, in_channels, out_channels]
kernels = rng.random(size = (kr_ht, kr_wt, no_ch, no_kr))
kernels.shape

(4, 4, 3, 5)

In [8]:
images = train_images[:no_im,:,:,:]
images.shape

# plt.figure(figsize=(5, 10))
# for idx, img in enumerate(images):
#     plt.subplot(5,2,idx+1)
#     plt.imshow(img)

(10, 32, 32, 3)

In [9]:
# tensorflow built-in method
tf_conv_out = tf.nn.conv2d(input=images,
                            filters=kernels,
                            strides=[1,1,1,1],
                            padding="VALID",
                            data_format='NHWC',
                            dilations=None,
                            name=None
                        )
tf_conv_out.shape

TensorShape([10, 29, 29, 5])

In [10]:
# manual convolution
# extract image patches
patches = tf.image.extract_patches(images=images,
                                 sizes=[1, kr_ht, kr_wt, 1],
                                 strides=[1, 1, 1, 1],
                                 rates=[1, 1, 1, 1],
                                 padding='VALID')
assert patch_len == patches.shape[-1]
assert no_of_patches == patches.shape[1]*patches.shape[2]

patches.shape

TensorShape([10, 29, 29, 48])

In [11]:
# flatten patches
flat_patches = tf.reshape(patches, (no_im, no_of_patches, patch_len))
# tranpose for matrix multiplication
flat_patches = tf.transpose(flat_patches, (0,2,1))
flat_patches.shape

TensorShape([10, 48, 841])

In [12]:
# flatten kernels
## first reorder kernels by no. of output-kernels
flat_kernels = tf.transpose(kernels, perm=(3,0,1,2))
flat_kernels = tf.reshape(flat_kernels, (no_kr, kr_ht*kr_wt*no_ch))
flat_kernels = tf.broadcast_to(flat_kernels, (no_im, no_kr, kr_ht*kr_wt*no_ch))
flat_kernels.shape

TensorShape([10, 5, 48])

In [13]:
manual_conv = flat_kernels@flat_patches
manual_conv.shape

TensorShape([10, 5, 841])

In [14]:
manual_conv = tf.transpose(manual_conv, (0,2,1))

In [15]:
manual_conv = tf.reshape(manual_conv, (no_im, y_ht,y_wt, no_kr))
manual_conv.shape

TensorShape([10, 29, 29, 5])

In [16]:
tf.reduce_sum(tf_conv_out - manual_conv)

<tf.Tensor: shape=(), dtype=float64, numpy=1.1370904218210853e-12>